In [1]:
!pip install Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
import sys
from matplotlib import pyplot
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
from keras.callbacks import ModelCheckpoint
import pandas as pd
from keras.layers.normalization import BatchNormalization
# from tensorflow.keras.callbacks import TensorBoard
from time import time
import keras
from keras.callbacks import TensorBoard

You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [21]:
TensorBoard

keras.callbacks.tensorboard_v1.TensorBoard

In [23]:
!pip install tensorboard==1.13.0

     |████████████████████████████████| 3.2MB 7.1MB/s eta 0:00:01
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
img_width, img_height = 224, 224
nb_train_samples = 539231
nb_validation_samples = 240424
epochs = 10
batch_size = 100
n_classes = 1

In [3]:
train_data_dir = "../../../../../raid/Data/Master_Dataset/mix/train"
validation_data_dir = "../../../../../raid/Data/Master_Dataset/mix/validation"

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode = 'constant',
    cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 539231 images belonging to 2 classes.
Found 240424 images belonging to 2 classes.


In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess) 

In [ ]:
base_model = vgg16.VGG16(input_shape=(img_width, img_height, 3),
                                     weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",
                                     include_top=False,
                                     pooling='avg')

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = Dense(n_classes, activation='sigmoid', name = 'prediction')(x)
model = Model(inputs = base_model.input, outputs = x)



In [ ]:
print(model.summary())

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto',min_delta=1e-2, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=1, mode='auto',verbose=1, min_delta=1e-2, min_lr=0.00001)
        

In [30]:
i = 0
class LossHistory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        global i
        self.model.save("paper/weights_of_epoch_" + str(i) + ".hdf5")
#         summarize_diagnostics(self.model_history, i)
        i += 1

In [31]:
e = 10
# model = tf.keras.models.load_model('/home/vision/mrunal/detect_deepfake/densenet121/paper/weights_of_epoch_0.hdf5')
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model = multi_gpu_model(model, gpus=2)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
final_location = []
lh = LossHistory()

tensorboard = TensorBoard(log_dir = "/home/vision/mrunal/detect_deepfake/vgg_logs/{}".format(time()))
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [tensorboard, lh, early_stop, reduce_lr]

model_history = model.fit_generator(
train_generator,
epochs=e,
steps_per_epoch = nb_train_samples // train_generator.batch_size,
validation_data=validation_generator,
verbose=1,
validation_steps=nb_validation_samples // validation_generator.batch_size,
callbacks=callbacks_list)

#model.save("weights_best_" + str(e) + ".hdf5")

# _,acc=model.evaluate_generator(validation_generator, steps=nb_validation_samples, max_queue_size=10,verbose=1, workers=1, use_multiprocessing=False)
# print('> %.3f' % (acc * 100.0))

loss=model_history.history['loss']
acc=model_history.history['accuracy']
valacc=model_history.history['val_accuracy']
valloss=model_history.history['val_loss']
location = [e,loss,acc,valacc, valloss]
final_location.append(location)
save = pd.DataFrame(final_location,columns=['epochs','loss','acc','valacc','valloss'])
save.to_csv('/home/vision/mrunal/detect_deepfake/VGG/paper/VGG16_log_'+str(e)+'.csv')

Epoch 1/10
5392/5392 [==============================] - 6689s 1s/step - loss: 0.2448 - accuracy: 0.8892 - val_loss: 0.6195 - val_accuracy: 0.8458
Epoch 2/10
5392/5392 [==============================] - 6379s 1s/step - loss: 0.0907 - accuracy: 0.9640 - val_loss: 0.6267 - val_accuracy: 0.8613
Epoch 3/10
5392/5392 [==============================] - 6289s 1s/step - loss: 0.0610 - accuracy: 0.9762 - val_loss: 0.5050 - val_accuracy: 0.8674

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 4/10
5392/5392 [==============================] - 6117s 1s/step - loss: 0.0339 - accuracy: 0.9868 - val_loss: 1.0333 - val_accuracy: 0.8529

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 5/10
5392/5392 [==============================] - 5949s 1s/step - loss: 0.0281 - accuracy: 0.9892 - val_loss: 0.5547 - val_accuracy: 0.8520

Epoch 00005: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 6/10
5392/5392 [===================